# Overview
Alright, so I was bored one night and had an interview comming up. 
A few months ago, a friend of mine raged about how shockingly few people were able to solve this problem durng interview:

> Given a list of a class that contains 3 fields
>
> Group the list by one field, sort the list by another, and return the top n of each group

The problem isn't difficult (though it's _plagued_ by ambiguity, which in my opinion makes this question a great interview question because it really lets you see how a candidate approaches an ambiguous problem--which in my opinion is incredibly valuable) but I found it to be a good excersize to get my mind back in "interview mode" and brush up on a bit of my python...

_Note: I'm not a python expect--though I really like python. While I do my best to write clean code, I'm honestly not going to bother cleaning up code that will likely have no audience besides maybe a couple students looking up how to implement a doubly linked list in python_

# Setting things up!
I created a class called myData (great name, right?) which contains the three values as listed in the problem statement. 
I added implementations required for sorting to work as well....

In [73]:
class myData :
    def __init__(self, val_0, val_sort_by, val_group_by) :
        self.val_0 = val_0
        self.val_sort_by = val_sort_by
        self.val_group_by = val_group_by
        
    def __gt__(self, other) :
        return self.val_sort_by > other.val_sort_by
    
    def __lt__(self, other) : 
        return self.val_sort_by < other.val_sort_by
    
    def print(self) :
        print("Group: " + str(self.val_group_by) + " Sort: " + str(self.val_sort_by) + " Val: " + str(self.val_0))

## Data Generation
To generate my data, I used 4 groups consisting of 4000 random values each to be used for sorting. The third value doesn't matter with how I interpreted the problem statement

In [168]:
import random

data = list()
for group in range(0,4) : 
    for sort_by in range(0,40000) : 
        data.append(myData(0,random.randint(0,10000000),group))
            
random.shuffle(data)

# First Implementation : Dictionary of Lists
Dictionaries are perfect fit to meet the grouping requirement.
In this implementation, I create a dictionary where the key is the group value and each instance of myData is added into the list with it's corresponding group key.
After the values are grouped, each list is sorted. 

While this method works, it's not very scalable. 
First, appending to the list has a constant time complexity O(1)
However, sorting is a different story. The time complexity here is O(n log n) -- not good... 

In [169]:
import time

data_dictionary = {}

start_time = time.time()
for datum in data :
    if datum.val_group_by not in data_dictionary :
        data_dictionary[datum.val_group_by] = []
    data_dictionary[datum.val_group_by].append(datum)

# Slow way of sorting
for key in data_dictionary : 
    data_dictionary[key].sort();

end_time = time.time()
print("Execution Time: " + str(end_time - start_time))

Execution Time: 0.9294300079345703


# Second Implementation : Doubly Linked List
The obvious solution to make this run a bit faster is to be able to sort in place. To acheive this, I implemented a doubly linked list with pointers to the current node containing the maximum number and the current node containing the minimum number. 
With the use of these pointers, no searching is required, resulting in a linear time complexity to be acheived!

In [170]:
class doubly_ll_node : 
    def __init__(self, next_node, prev_node, data):
        self.next_node = next_node
        self.prev_node = prev_node
        self.data = data
    
    def AddPrevious(self, node) : 
        self.prev_node = node;
    
    def AddNext(self, node) :
        self.next_node = node;
        
    def display(self) : 
        self.data.print()
        if self.next_node is not None :
            self.next_node.display()

class doubly_ll :
    def __init__(self) :
        self.first_node = None
        self.last_node = None
        self.first_node_added = None 
        
    def add(self, data) :
        if self.first_node is None or data.val_sort_by < self.first_node.data.val_sort_by : 
            self.add_first(data)
        elif self.last_node is None or data.val_sort_by >= self.last_node.data.val_sort_by :
            self.add_last(data)
        
    def add_first(self, data):
        if self.first_node is None :
            if self.first_node_added is None : 
                self.first_node = doubly_ll_node(None, None, data)
                self.first_node_added = self.first_node
            else :
                self.first_node = doubly_ll_node(self.first_node_added, None, data)
                self.first_node_added.AddPrevious(self.first_node)
        else :
            new_node = doubly_ll_node(self.first_node,None, data)
            self.first_node.AddPrevious(new_node);
            self.first_node = new_node; 
    
    def add_last(self, data):
        if self.last_node is None :
            if self.first_node_added is None : 
                self.last_node = doubly_ll_node(None, None, data)
                self.first_node_added = self.last_node
            else : 
                self.last_node = doubly_ll_node(None, self.first_node_added, data)
                self.first_node_added.AddNext(self.last_node)
                
        else : 
            new_node = doubly_ll_node(None, self.last_node, data)
            self.last_node.AddNext(new_node);
            self.last_node = new_node
            
    def print_first_n(self, n) :
        n_copy = n
        node_to_print = self.first_node
        while n_copy > 0 :
            if node_to_print is None : 
                return
            node_to_print.data.print()
            node_to_print = node_to_print.next_node
            n_copy = n_copy - 1
    
    def print(self) :
        if self.first_node is None : 
            self.last_node.display()
        else :
            self.first_node.display()

In [172]:
data_dictionary = {}

start_time = time.time()
for datum in data :
    if datum.val_group_by not in data_dictionary :
        data_dictionary[datum.val_group_by] = doubly_ll()
    data_dictionary[datum.val_group_by].add(datum);

end_time = time.time()
print("Execution Time: " + str(end_time - start_time))

for key in data_dictionary : 
    data_dictionary[key].print_first_n(3)

Execution Time: 0.21318912506103516
Group: 2 Sort: 81 Val: 0
Group: 2 Sort: 532 Val: 0
Group: 2 Sort: 1321 Val: 0
Group: 0 Sort: 273 Val: 0
Group: 0 Sort: 501 Val: 0
Group: 0 Sort: 1049 Val: 0
Group: 1 Sort: 203 Val: 0
Group: 1 Sort: 331 Val: 0
Group: 1 Sort: 4191 Val: 0
Group: 3 Sort: 14 Val: 0
Group: 3 Sort: 51 Val: 0
Group: 3 Sort: 476 Val: 0


# O